In [1]:
import argparse
import sys
import re
import os, glob, datetime, time
import numpy as np
import torch
import torch.nn as nn
from torch.nn.modules.loss import _Loss
import torch.nn.init as init
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import MultiStepLR
import data_generator as dg
from data_generator import DenoisingDataset
import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
# Params
if any(["jupyter" in arg for arg in sys.argv]):
    # Simulate command line arguments (replace these with your desired defaults)
    sys.argv = ['ipykernel_launcher.py', '--model', 'DnCNN', '--batch_size', '4', '--train_data', 'data/Train400', '--sigma', '4', '--epoch', '2', '--lr', '0.0005']

parser = argparse.ArgumentParser(description='PyTorch DnCNN')
parser.add_argument('--model', default='DnCNN', type=str, help='choose a type of model')
parser.add_argument('--batch_size', default=4, type=int, help='batch size')
parser.add_argument('--train_data', default='data/Train400', type=str, help='path of train data')
parser.add_argument('--sigma', default=4, type=int, help='noise level')
parser.add_argument('--epoch', default=2, type=int, help='number of train epoches')
parser.add_argument('--lr', default=5e-4, type=float, help='initial learning rate for Adam')
args = parser.parse_args()
batch_size = args.batch_size
cuda = torch.cuda.is_available()
n_epoch = args.epoch
sigma = args.sigma

save_dir = os.path.join('models', args.model+'_' + 'sigma' + str(sigma))

if not os.path.exists(save_dir):
    os.mkdir(save_dir)

class SE_block(nn.Module):
    def __init__(self, n_channels=64, reduction=8):
        super(SE_block, self).__init__()
        self.red = reduction
        self.adavg = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(n_channels, n_channels//self.red, bias=True)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(n_channels//self.red, n_channels, bias=True)
        self.sigm = nn.Sigmoid()
        
    def forward(self, x):
        b, c, h, w = x.size()
        y = self.adavg(x)
        y = y.view(b, c)
        y = self.relu(self.fc1(y))
        y = self.sigm(self.fc2(y)).view(b, c, 1, 1)
        return x * y.expand_as(x)


class Residual_Blocks(nn.Module):
    def __init__(self, in_channels=64, out_channels=64, padding = 1):
        super(Residual_Blocks, self).__init__()
        self.convx_1 = nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False)
        self.Leakyrelu = nn.LeakyReLU(inplace=True)
        self.BN = nn.BatchNorm2d(64)
        self.convx_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1, bias=False)
        self.Leakyrelu = nn.LeakyReLU(inplace=True)
        self.seblock = SE_block()

    def forward(self, x):
        x = self.BN(self.Leakyrelu(self.convx_1(x)))
        out = self.BN(self.Leakyrelu(self.convx_2(x)))
        return self.seblock(out)

        
class DnCNN(nn.Module):
    def __init__(self, n_channels=64, image_channels=1):
        super(DnCNN, self).__init__()
        self.conv1 = nn.Conv2d(image_channels, n_channels, kernel_size=3, padding=1, bias=False)
        self.act = nn.LeakyReLU(inplace=True)
        self.out = nn.Sequential(*[Residual_Blocks() for _ in range(0, 10)])
        self.dn = nn.Conv2d(64, 1, kernel_size=3, padding=1, bias=True)
        self._initialize_weights()
        
    def forward(self, x):
        y = x
        out = self.act(self.conv1(x))
        out = self.out(out)
        dn = y - self.dn(out)
        return dn
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.orthogonal_(m.weight)
                print('init weight')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)

                
class sum_squared_error(_Loss):  # PyTorch 0.4.1
    """
    Definition: sum_squared_error = 1/2 * nn.MSELoss(reduction = 'sum')
    The backward is defined as: input-target
    """
    def __init__(self, size_average=None, reduce=None, reduction='sum'):
        super(sum_squared_error, self).__init__(size_average, reduce, reduction)

    def forward(self, input, target):
        # return torch.sum(torch.pow(input-target,2), (0,1,2,3)).div_(2)
        return torch.nn.functional.mse_loss(input, target, size_average=None, reduce=None, reduction='sum').div_(2)


def log(*args, **kwargs):
     print(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S:"), *args, **kwargs)

In [4]:
import torch

# Attempt to manually select a CUDA device
try:
    torch.cuda.current_device()
    print("Current CUDA device:", torch.cuda.get_device_name())
except Exception as e:
    print("Error accessing CUDA device:", str(e))

Current CUDA device: NVIDIA GeForce GTX 1080 Ti


In [5]:
from tqdm import tqdm

if __name__ == '__main__':
    # model selection
    print('===> Building model')
    model = DnCNN()
    
    initial_epoch = 0
    model.train()
    # criterion = nn.MSELoss(reduction = 'sum')  # PyTorch 0.4.1
    criterion = sum_squared_error()
    if cuda:
        model = model.cuda()
         # device_ids = [0]
         # model = nn.DataParallel(model, device_ids=device_ids).cuda()
         # criterion = criterion.cuda()
    optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay = 1e-4)
    scheduler = MultiStepLR(optimizer, milestones=[3, 6, 9], gamma=0.2)  # learning rates
    for epoch in tqdm(range(initial_epoch, n_epoch)):

        scheduler.step(epoch)  # step to the learning rate in this epcoh
        xs = dg.datagenerator(data_dir=args.train_data)
        xs = xs.astype('float32')/255.0
        xs = torch.from_numpy(xs.transpose((0, 3, 1, 2)))  # tensor of the clean patches, NXCXHXW
        DDataset = DenoisingDataset(xs, sigma)
        DLoader = DataLoader(dataset=DDataset, num_workers=4, drop_last=True, batch_size=batch_size, shuffle=True)
        epoch_loss = 0
        start_time = time.time()

        for n_count, batch_yx in enumerate(DLoader):
                optimizer.zero_grad()
                if cuda:
                    batch_x, batch_y = batch_yx[1].cuda(), batch_yx[0].cuda()
                loss = criterion(model(batch_y), batch_x)
                epoch_loss += loss.item()
                loss.backward()
                optimizer.step()
                if n_count % 10 == 0:
                    print('%4d %4d / %4d loss = %2.4f' % (epoch+1, n_count, xs.size(0)//batch_size, loss.item()/batch_size))
        elapsed_time = time.time() - start_time

        log('epcoh = %4d , loss = %4.4f , time = %4.2f s' % (epoch+1, epoch_loss/n_count, elapsed_time))
        np.savetxt('train_result.txt', np.hstack((epoch+1, epoch_loss/n_count, elapsed_time)), fmt='%2.4f')
        # torch.save(model.state_dict(), os.path.join(save_dir, 'model_%03d.pth' % (epoch+1)))
        torch.save(model, os.path.join(save_dir, 'model_%03d.pth' % (epoch+1)))


===> Building model
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight
init weight


  0%|          | 0/2 [00:00<?, ?it/s]

^_^-training data finished-^_^
   1    0 / 43648 loss = 237.1547
   1   10 / 43648 loss = 88.6582
   1   20 / 43648 loss = 53.8974
   1   30 / 43648 loss = 30.7138
   1   40 / 43648 loss = 17.0435
   1   50 / 43648 loss = 7.8193
   1   60 / 43648 loss = 4.4998
   1   70 / 43648 loss = 3.2746
   1   80 / 43648 loss = 1.4488
   1   90 / 43648 loss = 1.7508
   1  100 / 43648 loss = 1.4478
   1  110 / 43648 loss = 0.9673
   1  120 / 43648 loss = 0.9431
   1  130 / 43648 loss = 0.6466
   1  140 / 43648 loss = 0.6453
   1  150 / 43648 loss = 0.5349
   1  160 / 43648 loss = 0.6615
   1  170 / 43648 loss = 0.5260
   1  180 / 43648 loss = 0.4872
   1  190 / 43648 loss = 0.4893
   1  200 / 43648 loss = 0.4112
   1  210 / 43648 loss = 0.3736
   1  220 / 43648 loss = 0.4402
   1  230 / 43648 loss = 0.4973
   1  240 / 43648 loss = 0.3926
   1  250 / 43648 loss = 0.4771
   1  260 / 43648 loss = 0.4213
   1  270 / 43648 loss = 0.3859
   1  280 / 43648 loss = 0.3470
   1  290 / 43648 loss = 0.3469
   

 50%|█████     | 1/2 [15:28<15:28, 928.62s/it]

^_^-training data finished-^_^
   2    0 / 43648 loss = 0.1906
   2   10 / 43648 loss = 0.1991
   2   20 / 43648 loss = 0.2015
   2   30 / 43648 loss = 0.1929
   2   40 / 43648 loss = 0.1942
   2   50 / 43648 loss = 0.1967
   2   60 / 43648 loss = 0.1969
   2   70 / 43648 loss = 0.1988
   2   80 / 43648 loss = 0.2031
   2   90 / 43648 loss = 0.1987
   2  100 / 43648 loss = 0.1940
   2  110 / 43648 loss = 0.2021
   2  120 / 43648 loss = 0.1988
   2  130 / 43648 loss = 0.1959
   2  140 / 43648 loss = 0.1899
   2  150 / 43648 loss = 0.2015
   2  160 / 43648 loss = 0.1955
   2  170 / 43648 loss = 0.1930
   2  180 / 43648 loss = 0.1929
   2  190 / 43648 loss = 0.1925
   2  200 / 43648 loss = 0.2000
   2  210 / 43648 loss = 0.2000
   2  220 / 43648 loss = 0.1973
   2  230 / 43648 loss = 0.1929
   2  240 / 43648 loss = 0.1929
   2  250 / 43648 loss = 0.1926
   2  260 / 43648 loss = 0.1896
   2  270 / 43648 loss = 0.1975
   2  280 / 43648 loss = 0.1940
   2  290 / 43648 loss = 0.2005
   2  300

100%|██████████| 2/2 [30:51<00:00, 925.61s/it]

2024-11-23 05:22:47: epcoh =    2 , loss = 0.7876 , time = 917.63 s
